# SVM model

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.svm import SVR

In [3]:
df = pd.read_csv('../Data/hans_model_data.csv')
df.head()

,Unnamed: 0,company,title,totalyearlycompensation,yearsofexperience,yearsatcompany,tag,year,month,year_month,year_month_3mos,state_short,inflation_rate,inflation_rate_3mos,employment_rate,employment_rate_3mos
0,0,Yahoo,Software Engineer,160.5,0.58,0.58,Full Stack,2018,6,2018-06,2018-03,CA,0.029,0.024,0.957660,0.956797
1,1,Facebook,Software Engineer,165.0,1.00,1.00,Full Stack,2018,6,2018-06,2018-03,WA,0.029,0.024,0.955998,0.954978
2,2,VmWare,Software Engineer,218.0,8.00,1.00,Distributed Systems (Back-End),2018,6,2018-06,2018-03,CA,0.029,0.024,0.957660,0.956797
3,3,Uber,Software Engineer,240.0,3.00,0.00,Web Development (Front-End),2018,6,2018-06,2018-03,CA,0.029,0.024,0.957660,0.956797
4,4,Capital One,Software Engineer,196.0,8.00,2.00,iOS,2018,6,2018-06,2018-03,NY,0.029,0.024,0.959053,0.955962


In [ ]:
df.drop(columns = ['Unnamed: 0', 'timestamp', 'location', 'timestamp_3mos', 'state'], inplace = True)
df.drop(columns = ['level'], inplace = True)

In [4]:
df.drop(columns = ['Unnamed: 0', 'tag','year_month', 'year_month_3mos'], inplace = True)

In [5]:
df.columns

Index(['company', 'title', 'totalyearlycompensation', 'yearsofexperience',
       'yearsatcompany', 'year', 'month', 'state_short', 'inflation_rate',
       'inflation_rate_3mos', 'employment_rate', 'employment_rate_3mos'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24496 entries, 0 to 24495
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company                  24496 non-null  object 
 1   title                    24496 non-null  object 
 2   totalyearlycompensation  24496 non-null  float64
 3   yearsofexperience        24496 non-null  float64
 4   yearsatcompany           24496 non-null  float64
 5   year                     24496 non-null  int64  
 6   month                    24496 non-null  int64  
 7   state_short              24496 non-null  object 
 8   inflation_rate           24496 non-null  float64
 9   inflation_rate_3mos      24496 non-null  float64
 10  employment_rate          24496 non-null  float64
 11  employment_rate_3mos     24496 non-null  float64
dtypes: float64(7), int64(2), object(3)
memory usage: 2.2+ MB


In [7]:
X = df[['company', 'title', 'yearsofexperience', 'yearsatcompany', 'year', 'month', 'state_short', 'inflation_rate',
            'inflation_rate_3mos', 'employment_rate', 'employment_rate_3mos']].copy()
X_dummy = pd.get_dummies(X, columns = ['company', 'title', 'state_short'], drop_first = True)

y = df[['totalyearlycompensation']].copy()

X_dummy.head()

,yearsofexperience,yearsatcompany,year,month,inflation_rate,inflation_rate_3mos,employment_rate,employment_rate_3mos,company_ Credit Karma,company_23andMe,...,state_short_RI,state_short_SC,state_short_TN,state_short_TX,state_short_UT,state_short_VA,state_short_VT,state_short_WA,state_short_WI,state_short_WV
0,0.58,0.58,2018,6,0.029,0.024,0.957660,0.956797,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.00,1.00,2018,6,0.029,0.024,0.955998,0.954978,0,0,...,0,0,0,0,0,0,0,1,0,0
2,8.00,1.00,2018,6,0.029,0.024,0.957660,0.956797,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.00,0.00,2018,6,0.029,0.024,0.957660,0.956797,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8.00,2.00,2018,6,0.029,0.024,0.959053,0.955962,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_dummy, y, test_size = 0.2, random_state = 42)

In [9]:
#Feature Scaling

sc = StandardScaler()

X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [10]:
# Fitting SVR to the dataset

svr = SVR()
svr.fit(X_train_sc, y_train)

/Users/hansbaumberger/opt/anaconda3/envs/dsi-111/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVR()

In [11]:
# Predictions
y_preds = svr.predict(X_test)

# Measure performance based on accuracy.
print('train score:', svr.score(X_train_sc, y_train))
print('test score:', svr.score(X_test, y_test))

train score: 0.13682281863731038
test score: -0.4287102177581459


In [12]:
# Gridsearch:

params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf'],
    'gamma': [0.001, 0.0001]
}

model = SVR()

#Gridsearch:
gs = GridSearchCV(model, params, n_jobs=-1, verbose=1)

# Fit on training data.
gs.fit(X_train_sc, y_train);

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/hansbaumberger/opt/anaconda3/envs/dsi-111/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [13]:
gs.best_params_

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

In [14]:
# Accuracy
print(f'Training acc: {gs.score(X_train_sc, y_train)}')
print(f'Testing acc: {gs.score(X_test_sc, y_test)}')

Training acc: 0.5029896322968699
Testing acc: 0.47450456180594414


In [17]:
# Creating Predictions
#train_preds = gs.predict(X_train_sc)
#test_preds = gs.predict(X_test_sc)

In [18]:
# DataFrame with column for predicted values.
#results = pd.DataFrame(gs.predict(X_test), columns=['predicted'])

# Column for observed values.
#results['actual'] = y_test

#results.head()

# NN model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model_es = Sequential()

n_input = X_train_sc.shape[1]

model_es.add(Dense(17, input_dim=n_input, activation='relu'))
model_es.add(Dense(10, activation='relu'))
model_es.add(Dense(1))

model_es.compile(loss='mse', optimizer='adam', metrics=['mae', 'acc'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')

history_es = model_es.fit(
    X_train,
    y_train,
    validation_data=(X_test_sc, y_test),
    epochs=100,
    batch_size=None,
    callbacks=[early_stop]
)

In [ ]:
model.predict(X_test_sc)

In [ ]:
model.evaluate(X_test_sc, y_test) # kind of like .score() in sklearn

In [ ]:
from tensorflow.keras.regularizers import l2

model_l2 = Sequential()

n_input = X_train.shape[1]

model_l2.add(Dense(
    17,
    input_dim=n_input,
    activation='relu',
    kernel_regularizer=l2(0.001)
))

model_l2.add(Dense(
    10,
    activation='relu',
    kernel_regularizer=l2(0.001)
))

model_l2.add(Dense(
    1,
    kernel_regularizer=l2(0.001)
))

model_l2.compile(
    loss='mse',
    optimizer='adam',
    metrics=['acc', 'mae']
)

history_l2 = model_l2.fit(
    X_train_sc,
    y_train,
    validation_data=(X_test_sc, y_test),
    epochs=100,
    batch_size=None
)

In [ ]:
model.predict(X_test_sc)

In [ ]:
model.evaluate(X_test_sc, y_test) # kind of like .score() in sklearn

In [ ]:
from tensorflow.keras.layers import Dropout

model_dropout = Sequential()

n_input = X_train.shape[1]
n_hidden = n_input

model_dropout.add(Dense(n_hidden, input_dim=n_input, activation='relu'))
model_dropout.add(Dropout(0.2)) # refers to nodes in the first hidden layer
model_dropout.add(Dense(10, activation='relu'))
model_dropout.add(Dropout(0.2))
model_dropout.add(Dense(1))

model_dropout.compile(loss='mse', optimizer='adam', metrics=['acc', 'mae'])

history_dropout = model_dropout.fit(
    X_train_sc,
    y_train,
    validation_data=(X_test_sc, y_test), 
    epochs=100,
    batch_size=None
)

In [ ]:
model.predict(X_test_sc)

In [ ]:
model.evaluate(X_test_sc, y_test) # kind of like .score() in sklearn